<a href="https://colab.research.google.com/github/compartia/nlp_tools/blob/tensorflow-model-tokens/Creating_fake_contracts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
GLOBALS__={}

# Create fake contracts to train structure detector

Sentencepiece Vacabulary t is build with 
(https://github.com/compartia/nlp_tools/blob/tensorflow-model-tokens/Build_wordpieces_dictionary.ipynb)

## MAIN, init, load code

In [66]:
#@title  { form-width: "300px", display-mode: "form" }
import os

!pip install sentencepiece
import sentencepiece as spm




# ''' AZ:-IMPORT CODE GITHUB----------------------------------------------😺------ '''
import sys
# ====================================
# ====================================
_git_branch = "tensorflow-model-tokens"  # @param {type:"string"}
# ====================================
# ====================================


def _init_import_code_from_gh():
  if 'GLOBALS__' not in globals():
    print('adding global GLOBALS__')
    global GLOBALS__
    GLOBALS__ = {}

  if '_init_import_code_from_gh' in GLOBALS__:
    print('👌 code already imported from GitHub!')
    return

  import subprocess
  def exec(x):
    r = subprocess.check_output(x, shell=True)
    r = r.decode('unicode-escape').encode('latin1').decode('utf8')
    print(r)

  print(f"fetching code from GitHub.....{_git_branch}")
  try:
    exec('rm -r nlp_tools')
  except:
    pass
  exec(f'git clone --single-branch --branch {_git_branch} https://github.com/compartia/nlp_tools.git nlp_tools')

  print('🦊 GIT revision:')
  exec('cd nlp_tools\ngit rev-list --reverse HEAD | awk "{ print NR }" | tail -n 1\ngit branch\ngit log -3 --pretty=%B')

  sys.path.insert(0, 'nlp_tools')

  # self-test

  print('installing antiword...')
  exec('sudo apt-get install antiword')

  print('installing docx2txt...')
  exec("pip install docx2txt")

  GLOBALS__['_init_import_code_from_gh'] = True

  ''' AZ:-------------------------------------------------IMPORT CODE GITHUB-😺---'''
  print('❤️ DONE importing Code fro GitHub')


# AZ:-INIT ELMO-----------------------------------------------------------------------------------

import tensorflow_hub as hub

#
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML


 

# AZ:- THE CODE----------------------------------------------------------------------------------

def _init_the_code(reset=False):
  if '_init_the_code' in GLOBALS__ and not reset:
    print('👌 Code is alredy imported!')
    return

  from renderer import HtmlRenderer
  from renderer import to_multicolor_text
  from renderer import known_subjects_dict

  from structures import ContractSubject
  from contract_parser import ContractDocument3

  from ml_tools import ProbableValue

  from legal_docs import LegalDocument
  from renderer import as_warning, as_headline_3, as_offset, as_smaller

  class DemoRenderer(HtmlRenderer):

    def render_color_text(self, tokens, weights, colormap='coolwarm', print_debug=False, _range=None):
      html = self.to_color_text(tokens, weights, colormap, print_debug, _range)
      display(HTML(html))

    def to_color_text(self, tokens, weights, colormap='coolwarm', print_debug=False, _range=None):
      return super()._to_color_text(tokens, weights, mpl, colormap=colormap, _range=_range)

    def render_multicolor_text(self, tokens, vectors, colormap, min_color=None, _slice=None):
      display(HTML(to_multicolor_text(tokens, vectors, colormap, min_color=min_color, _slice=_slice)))

     
     

     
  GLOBALS__['renderer'] = DemoRenderer()

  GLOBALS__['_init_the_code'] = True
  print("❤️ DONE initializing the code")

  # AZ:-------------------------------------------------Init Protocols context===


def read_doc(fn):
  import docx2txt, sys, os

  text = ''
  try:
    text = docx2txt.process(fn)

  except:
    print("Unexpected error:", sys.exc_info())
    os.system('antiword -w 0 "' + fn + '" > "' + fn + '.txt"')
    with open(fn + '.txt') as f:
      text = f.read()

  return text

def interactive_upload(filetype):
  from google.colab import files
  import docx2txt

  print(f'Please select "{filetype}" .docx file:')
  uploaded = files.upload()
  docs = []
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

    with open(fn, "wb") as df:
      df.write(uploaded[fn])
      df.close()

    # extract text

    text = ''
    try:
      text = docx2txt.process(fn)
    except:
      print("Unexpected error:", sys.exc_info())
      os.system('antiword -w 0 "' + fn + '" > "' + fn + '.txt"')
      with open(fn + '.txt') as f:
        text = f.read()
    print("Символов в оригинальном документе:", len(text))
    docs.append(text)
    return docs

  
# AZ:- ENDO OF THE THE CODE------------------------------------------------XXXX


# 1.
_init_import_code_from_gh()
 
# 3.
_init_the_code(True)



fetching code from GitHub.....tensorflow-model-tokens


🦊 GIT revision:
472
* tensorflow-model-tokens
Update tokeniser

+ tokens_map
+ MarkedDoc
+ token_index_by_char_index
+ set_region_value_by_char_indices

Created using Colaboratory

installing antiword...
Reading package lists...
Building dependency tree...
Reading state information...
antiword is already the newest version (0.37-11build1).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.

installing docx2txt...

❤️ DONE importing Code fro GitHub
❤️ DONE initializing the code


### 📂👤Load files from GDrive

In [0]:

from google.colab import auth
from google.colab import drive
 

drive.mount('/content/gdrive', force_remount=True)


import glob
def read_documents(filename_prefix):
  texts = {}
  for file in glob.glob(filename_prefix+"*.doc"):
    try:
      text = read_doc(file)
      texts[file] = text
      print("good:", file)
    except:
      print('WRONG *.doc FILE!!', file)

  for file in glob.glob(filename_prefix+"*.docx"):
    try:
      text = read_doc(file)
      texts[file] = text
      print("good:", file)
    except:
      print('WRONG *.docx FILE!!', file)
      
  return texts

contracts_filename_prefix='/content/gdrive/My Drive/GazpromOil/Contracts/'
contracts = read_documents(contracts_filename_prefix)


assert len(contracts) > 0


## Fetching sample headlines from real contracts
- we going to mix smaple headlines with fake generated ones

In [0]:
def _normalize_headline(hl:str):
  tokens = hl.split(' ')
  result=''
  for s in tokens:
    if s.isalpha():
      result+=s+' '
  result=result.strip()
 
  return result


_headline_ok('317.1')

In [69]:
from contract_parser import ContractDocument3
import random



HEADLINES=[]
for c in contracts.values():
  _doc = ContractDocument3(c)
  _doc.parse()
  for s in _doc.structure.structure:
    if s.numbered and s.level<2:
#       _slice = slice(s.slice.start+s.text_offset + s.slice.stop)
      headline = s.to_string_no_number(_doc.tokens_cc, _doc.tokenizer).strip()
      headline = _normalize_headline(headline)
    
      if headline is not None and len(headline)<140 and len(headline)>8:
#         print (headline)
        HEADLINES.append(headline)
        HEADLINES.append(headline.upper())

len(HEADLINES)
random.sample(HEADLINES, k=1)[0] 

'Для выполнения демонтажных работ и вывоза демонтированного Движимого имущества обеспечить Покупателю доступ к месту нахождения'

In [0]:
HEADLINES

### Rendering utils

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns

_c_cyan1=(0, 1, 0.6)
_c_orange1=(1, 0.7, 0)
_c_green1=(0.3, 1, 0)
_c_pink1=(0.3, 0, 1)

_c_cyan2=(0, 0.8, 0.6)
_c_orange2=(1, 0.6, 0)
_c_green2=(0.4, 1, 0)
_c_pink2=(0.4, 0, 1)

token_colors=[
    (0.5, 0.5, 0.5),#undefined
    
    _c_cyan1,
    _c_orange1,
    _c_green1,
    _c_pink1,
    _c_pink1,
    
    _c_cyan2,
    _c_orange2,
    _c_green2,
    _c_pink2,
    _c_pink2,
    
    (0.1, 0.3, 1)#headlines
]

_colormap={}
for k in range(20):
  key = f'c{k}'
  _colormap[key] = token_colors[ k % len(token_colors) ]
    
    
def color_matrix(matrix, tokens):
    
  mt = matrix.T
  vectors = {}

  for k in range( int(matrix.shape[1])):

    key = f'c{k}'
    _colormap[key] = token_colors[ k % len(token_colors) ]
    vectors[key] = mt[k]
  
 
  GLOBALS__['renderer'].render_multicolor_text(tokens, vectors, _colormap, min_color=(0.4, 0.4, 0.4) )
  
  
  
  
  
##------------
def plot_matrix(matrix):
    
  mt = matrix.T
  
  with sns.axes_style("white"):
    plt.figure(figsize=(25, 3))
    ax = sns.heatmap(mt, square=False,  cmap="YlGnBu")
    plt.show()


# Creating fake contracts

In [0]:
import random


### Tokenizer

In [78]:
import sentencepiece_pb2
from documents import SpmGTokenizer, Tokens


tkz = SpmGTokenizer()
DICT_SIZE=tkz.sp.get_piece_size()
tokenizer=tkz

_str=' \na\na гонка a\nии  и\n\n  \n '
print(tkz.tokenize(_str))
print(tkz.sp.encode_as_pieces(_str))
print(tkz.sp.encode_as_ids(_str))


loading tokenization model /content/nlp_tools/vocab/m.model
['', '\n', 'a', '\n', 'a', ' ', 'го', 'н', 'ка', ' a', '\n', 'и', 'и', '  и', '\n', '\n', '  ', '\n']
['▁', '\n', 'a', '\n', 'a', '▁', 'го', 'н', 'ка', '▁a', '\n', 'и', 'и', '▁и', '\n', '\n', '▁', '\n']
[8, 6, 130, 6, 130, 8, 22, 51, 148, 226, 6, 12, 12, 13, 6, 6, 8, 6]


### Utils to generate random doc pieces

In [0]:
import random



def random_sentence(min_len=3, max_len=20, fullstop_p=0.99):
  length = random.randint(min_len, max_len)
  indices = random.choices(range(0, DICT_SIZE), k=length)
  line = tkz.sp.decode_ids(indices)

  # may be capitalize
  if random.random() < 0.9:
    line = line[0].upper() + line[1:]

  # may be full-stopped
  if random.random() < fullstop_p:
    line = line + "."

  return line


def random_structure(num, depth, capitals, numbered, parent=None):
  
  #TODO: 'Статья'
  
  lines = []

#   Статья
  
  for i in range(num):
    line = random.sample(HEADLINES, k=1)[0]
    if random.random() < 0.4:
      line = random_sentence(min_len=3, max_len=15, fullstop_p=0.5)

    # may be numbered
    number = None

    number = f'{i+1}'
    if parent:
      number = f'{parent}.{number}'

    if numbered and random.random() < 0.95:
      line = number + '. ' + line

    # may be uppercase
    if capitals or random.random() < 0.1:
      line = line.upper()

    lines.append((depth, line))

    if depth > 0 and random.random() < 0.7:
      sublines = random_structure(random.randint(0, 12//(depth+1)), depth - 1, False, numbered=True, parent=number)
      lines += sublines

  return lines

def _probably_new_line(t):
  if random.random() < 0.5:
    return t+'\n'
  return t

def random_paragraph():
  sentences_n = random.randint(1, 3)
  p = ''
  for c in range(sentences_n):
    s = random_sentence(min_len=10, max_len=50, fullstop_p=0.99)
    p += s
    p = _probably_new_line(p)
    p = _probably_new_line(p)
    
  return p


def _normalize_by_tokenization(line):
  lt = tokenizer.tokenize(line)
  return tokenizer.untokenize(lt)


      
def random_doc():
  headlines = random.randint(4, 15)
  headlines_depth = random.randint(0, 2)

  lines = random_structure(num=headlines, depth=headlines_depth, capitals=random.random() < 0.5,
                           numbered=random.random() < 0.8)
  doc = ''
  slices = []

  for (depth, line) in lines:    
    doc = _probably_new_line(doc)
    
    line = _normalize_by_tokenization(line)
        
    if depth == headlines_depth:
      
      doc = _probably_new_line(doc)
      doc = _probably_new_line(doc)
      end = len(doc)
      slices.append(slice(end, end + len(line)))

       
    doc += line
    doc += '\n'
    doc = _probably_new_line(doc)
    
    if random.random() < 0.2 or depth == headlines_depth:
      #add text
      doc += _normalize_by_tokenization( random_paragraph() )

    doc = _probably_new_line(doc)

  return doc, slices




In [80]:
import numpy as np

from documents import TokenizedText, TOKENIZER_DEFAULT, GTokenizer


# build tokens map to char pos
def tokens_map(_tokens):
  _text = ''
  _map = []
  for t in _tokens:
    _map.append(len(_text))
    _text += t
  _map.append(len(_text))
  return _map, _text


def token_index_by_char_index(char_index, _map):
  for i in range(len(_map) - 1):
    a = _map[i]
    b = _map[i + 1]

    if a <= char_index < b:
      return i

  return 0


class MarkedDoc(TokenizedText):

  def __init__(self, tokens, categories_vector=None, tokenizer: GTokenizer = TOKENIZER_DEFAULT):
    super().__init__(tokenizer)

    self.tokens = np.array(tokens)
    if categories_vector is None:
      categories_vector = np.zeros(len(tokens))

    assert len(tokens) == len(categories_vector)
    self.categories_vector = np.array(categories_vector)

    self.tokens_map, self.text = tokens_map(self.tokens)

  def filter(self, filter_op):
    new_tokens = []
    new_categories_vector = []

    for i in range(self.get_len()):
      _tuple = filter_op(self.tokens[i], self.categories_vector[i])

      if _tuple is not None:
        new_tokens.append(_tuple[0])
        new_categories_vector.append(_tuple[1])

    self.tokens = new_tokens
    self.categories_vector = new_categories_vector

  def slice(self, sl: slice):
    new_tokens = self.tokens[sl]
    new_categories_vector = self.categories_vector[sl]
    return MarkedDoc(new_tokens, new_categories_vector)

  def token_index_by_char_index(self, char_index):
    return token_index_by_char_index(char_index, self.tokens_map)

  def set_region_value_by_char_indices(self,  char_slice:slice, value ):
    a = token_index_by_char_index(char_slice.start, self.tokens_map)
    b = token_index_by_char_index(char_slice.stop, self.tokens_map)

    ts = slice(a, b)
    self.categories_vector[ts] = value
    
md = MarkedDoc(['dd', 'аа'])
md2 = md.slice(slice(0, 4))
md2.get_len()


2

In [81]:
 
doc_text, headline_slices = random_doc() 
doc = MarkedDoc(tkz.tokenize(doc_text), categories_vector=None, tokenizer=tkz)
for s in headline_slices:
  doc.set_region_value_by_char_indices(s, 1)

GLOBALS__['renderer'].render_color_text(doc.tokens, doc.categories_vector)


In [82]:

!pip install rstr

from trainset_builder import random_widow
import numpy as np
import rstr


def generate_trainset(num, segment_size):
  count=0
  while count<num:
    doc_text, headline_slices = random_doc() 
    doc = MarkedDoc(tkz.tokenize(doc_text), categories_vector=None, tokenizer=tkz)
    for s in headline_slices:
      doc.set_region_value_by_char_indices(s, 1)
    
    if doc.get_len() > segment_size:
      wnd = random_widow(segment_size, doc.get_len())
      trimmed = doc.slice ( wnd )
      assert trimmed.get_len() == segment_size      
      count+=1
      yield trimmed
  pass




#---------------------------------------
fragments = generate_trainset(2048, 1024)
#---------------------------------------



TRAINSET=[]
for trimmed in fragments:
  TRAINSET.append( (trimmed.tokens, trimmed.categories_vector) )

### 🏺 🥒 Pickle everything

In [0]:
import pickle

_path = '/content/gdrive/My Drive/GazpromOil/'

with open(_path+'doc_structure_trainset.pickle', 'wb') as handle:
  pickle.dump(TRAINSET, handle, protocol=pickle.HIGHEST_PROTOCOL)



### 🏺 🥒 Check what we have pickled

In [84]:
with open(_path+'doc_structure_trainset.pickle', 'rb') as handle:
  TRAINSET_read = pickle.load(handle)
for a, b in TRAINSET_read[0:10]:
  GLOBALS__['renderer'].render_color_text(a, b, _range=(0,1))